# Skywalker algorithm against the lcls mirror systems

In [1]:
SIMULATION = True
ALIGNMENT = 'HOMS'
#ALIGNMENT = 'MEC'
#ALIGNMENT = 'MFX'
#ALIGNMENT = 'XCS'

alignment_goals = dict(HOMS=[210, 270],
                       MEC=[270],
                       MFX=[270],
                       XCS=[270, 270])
first_steps = 10 # Naive first step for the parameters search
tolerances = 2.0 # Tolerance of each target in Pixels
average = 1 # Number of shots to average over
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [3]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level,
                format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')

from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker
from pswalker.plans import walk_to_pixel
from pswalker.watcher import Watcher
#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    mecy1 = system['mecy1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', x=0.0317324, z=101.843, alpha=0.0014)
    xrtm2 = mirror.OffsetMirror('test_xrtm2', x=0.0317324, z=200, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    mecy1 = pim.PIM('test_mecy1', x=0.0317324, z=350,
             zero_outside_yag= not infinite_yag)
    mfxdg1 = mecy1
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)
    patch_pims([mecy1], mirrors=[xrtm2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise
    mecy1.centroid_noise = centroid_noise

alignment_yags = dict(HOMS=[y1, y2],
                      MEC=[mecy1],
                      MFX=[mfxdg1],
                      XCS=['pbty', 'xcssb1y'])
alignment_mots = dict(HOMS=[m1, m2],
                      MEC=[xrtm2],
                      MFX=[xrtm2],
                      XCS=['xrtm1', 'xrtm3'])

2017-07-27 15:19:21,269 - INFO - pswalker.examples - Patching 2 pim(s)
2017-07-27 15:19:21,271 - INFO - pswalker.examples - Patching 1 pim(s)


In [4]:
#Create Skywalker plan
yags = alignment_yags[ALIGNMENT]
mots = alignment_mots[ALIGNMENT]
goals = alignment_goals[ALIGNMENT]
if SIMULATION:
    det_rbv = 'detector_stats2_centroid_x'
else:
    det_rbv = 'detector_stats2_centroid_y'
mot_rbv = 'pitch'
plan = skywalker(yags, mots, det_rbv, mot_rbv,
        goals, first_steps=first_steps, tolerances=tolerances,
        averages=average, timeout=10)  

In [5]:
#Create RunEngine
RE = RunEngine({})
RE.record_interruptions = True
#Subscribe a Watcher instance
watcher = Watcher()
RE.msg_hook = watcher
RE.subscribe('all',  watcher)


0

In [6]:
RE(plan)

2017-07-27 15:19:21,557 - INFO - pswalker.plans - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 2991.8140634994447
2017-07-27 15:19:21,559 - INFO - pswalker.callbacks - Unable to yield estimate from model Model(linear)
2017-07-27 15:19:21,560 - INFO - pswalker.plans - Walking motor, error after step #0 -> -2781.8140634994447
2017-07-27 15:19:21,562 - INFO - pswalker.plans - No model yielded accurate prediction, using naive plan
2017-07-27 15:19:21,680 - INFO - pswalker.plans - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 2969.9701682139307
2017-07-27 15:19:21,682 - INFO - pswalker.plans - Walking motor, error after step #1 -> -2759.9701682139307
2017-07-27 15:19:21,683 - INFO - pswalker.plans - Using model Model(linear) to determine next step.
2017-07-27 15:19:21,684 - INFO - pswalker.plans - Adjusting motor MIRR:TST:test_m1h to position 1273.4986527288938
2017-07-27 15:19:21,797 - INFO - pswalker.plans - Averaged data yielded TST:test_p3h_d

2017-07-27 15:19:24,111 - INFO - pswalker.plans - No model yielded accurate prediction, using naive plan
2017-07-27 15:19:24,112 - INFO - pswalker.plans - Using gradient of 46.000925248471304 for naive step...
2017-07-27 15:19:24,143 - INFO - pswalker.plans - ['TST:test_dg3_detector_stats2_centroid_x', 'MIRR:TST:test_m2h_pitch']
2017-07-27 15:19:24,144 - INFO - pswalker.plans - {'TST:test_dg3_detector_stats2_centroid_x': -943.81593963680211, 'MIRR:TST:test_m2h_pitch': 1428.3775283335783}
2017-07-27 15:19:24,276 - INFO - pswalker.plans - Averaged data yielded TST:test_dg3_detector_stats2_centroid_x is at 271.0970946915777
2017-07-27 15:19:24,278 - INFO - pswalker.plans - Succesfully walked to value 271.0970946915777 (target=270.0) after 1 steps.
2017-07-27 15:19:24,279 - INFO - pswalker.iterwalk - Found equation of (46.016570791377, -66672.16732969743) between linear fit of MIRR:TST:test_m2h to TST:test_dg3
2017-07-27 15:19:24,546 - INFO - pswalker.plans - Averaged data yielded TST:test

['a808ffce-0be1-4017-80b9-4f09be41c2d7']

In [7]:
#Print Run Statistics
watcher.report()


Skywalker successfuly aligned MIRR:TST:test_m1h, MIRR:TST:test_m2h to 210, 270
on TST:test_p3h, TST:test_dg3 in 0.0 seconds! The entire alignment moved the
mirrors 13 times, cycling between YAGs 9 times. The user requested that the
mirrors hit their targets within 2.0 pixels, averaging over 1 consecutive
images after each mirror motion. The resulting locations of mirrors and
centroid measurements can be seen in the table below.

+-----------------------------------------+---------------------+
|                                   Field | Last Measured Value |
+-----------------------------------------+---------------------+
| TST:test_p3h_detector_stats2_centroid_x |           209.98252 |
|                 MIRR:TST:test_m1h_pitch |          1457.56112 |
| TST:test_dg3_detector_stats2_centroid_x |           268.87275 |
|                 MIRR:TST:test_m2h_pitch |          1458.86686 |
+-----------------------------------------+---------------------+


'\nSkywalker successfuly aligned MIRR:TST:test_m1h, MIRR:TST:test_m2h to 210, 270\non TST:test_p3h, TST:test_dg3 in 0.0 seconds! The entire alignment moved the\nmirrors 13 times, cycling between YAGs 9 times. The user requested that the\nmirrors hit their targets within 2.0 pixels, averaging over 1 consecutive\nimages after each mirror motion. The resulting locations of mirrors and\ncentroid measurements can be seen in the table below.\n\n+-----------------------------------------+---------------------+\n|                                   Field | Last Measured Value |\n+-----------------------------------------+---------------------+\n| TST:test_p3h_detector_stats2_centroid_x |           209.98252 |\n|                 MIRR:TST:test_m1h_pitch |          1457.56112 |\n| TST:test_dg3_detector_stats2_centroid_x |           268.87275 |\n|                 MIRR:TST:test_m2h_pitch |          1458.86686 |\n+-----------------------------------------+---------------------+'